In [1]:
import pandas as pd
import requests
import io
import re
import numpy as np
import ccxt
pd.options.display.max_rows = 50

### Section I. Ingest data

In [3]:
df = pd.read_csv("Candles.csv")

#### separate out the months, days, years, and hours for more finer grained analysis

In [4]:
df = df.join(df['__time'].str.split(' ', expand=True).add_prefix('time'))

#### split the raw_pairs to find triangular trade opportunities

In [5]:
df = df.join(df['raw_pair'].str.split('/', expand=True).add_prefix('raw_pair'))

#### rename columns for easier readibility 

In [6]:
df.rename(columns ={'time0': 'weekday', 'time1': 'month', 'time2': 'day', 'time3':'year', 'time4':'time', 'time5':'24_hour_time'}, inplace =True)

#### the df contains pair trades and then other types of trades
#### separate out the pair trades from the non pair trades as the strategies will differ

In [8]:
mask = df.raw_pair.str.contains('/', na=False)

In [156]:
df_pairs = df[mask]
df_no_pairs = df[~mask]

In [157]:
df_no_pairs

,__time,marketplace,raw_pair,price_open,price_high,price_low,price_close,price_vwap,trade_count,volume,...,day,year,time,24_hour_time,time6,time7,time8,raw_pair0,raw_pair1,arb_opp
77,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,bitmex,ADAM19,0.000013,0.000013,0.000013,0.000013,0.000013,1346,11120476.0,...,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ADAM19,None,ADAM19 Fri Mar 15 2019 18:00:00 GMT-0600 (Moun...
78,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,bitmex,BCHH19,0.037600,0.039852,0.037600,0.039078,0.038859,13367,160716.0,...,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),BCHH19,None,BCHH19 Fri Mar 15 2019 18:00:00 GMT-0600 (Moun...
79,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,bitmex,BCHM19,0.038025,0.039863,0.038025,0.039118,0.039017,1287,9832.0,...,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),BCHM19,None,BCHM19 Fri Mar 15 2019 18:00:00 GMT-0600 (Moun...
80,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,bitmex,EOSH19,0.000950,0.000965,0.000946,0.000955,0.000956,5781,1681084.0,...,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),EOSH19,None,EOSH19 Fri Mar 15 2019 18:00:00 GMT-0600 (Moun...
81,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,bitmex,EOSM19,0.000958,0.000973,0.000953,0.000959,0.000962,1153,157611.0,...,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),EOSM19,None,EOSM19 Fri Mar 15 2019 18:00:00 GMT-0600 (Moun...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166720,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,deribit,BTC-PERPETUAL,61377.010000,61377.010000,59660.510000,60066.652000,60401.996000,53013,354854112.0,...,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),BTC-PERPETUAL,None,BTC-PERPETUAL Sat Mar 13 2021 17:00:00 GMT-070...
1166721,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,deribit,BTC-19MAR21,61829.656000,61902.130000,59963.310000,60364.402000,60945.312000,4442,14766160.0,...,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),BTC-19MAR21,None,BTC-19MAR21 Sat Mar 13 2021 17:00:00 GMT-0700 ...
1166722,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,deribit,BTC-26MAR21,62378.117000,62378.117000,60444.800000,60726.312000,61522.152000,20413,77718328.0,...,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),BTC-26MAR21,None,BTC-26MAR21 Sat Mar 13 2021 17:00:00 GMT-0700 ...
1166723,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,deribit,BTC-25JUN21,67044.790000,67110.450000,65311.027000,65612.850000,66005.140000,15193,28691750.0,...,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),BTC-25JUN21,None,BTC-25JUN21 Sat Mar 13 2021 17:00:00 GMT-0700 ...


In [10]:
df_no_pairs.to_csv('no_pairs.csv')
df_pairs.to_csv('pairs.csv')

In [159]:
#### exploratory analysis

In [31]:
time_pairs_df = df_pairs.groupby(['raw_pair0','raw_pair1'])

In [32]:
time_pairs_df.max().to_csv('grouped_by_ticker_max.csv')

In [34]:
time_pairs_df.min().to_csv('grouped_by_ticker_min.csv')

In [51]:
df_pairs.groupby(['__time','raw_pair0', 'raw_pair1']).count().to_csv('grouped_by_time_ticker_count.csv')

#### opportunities siloed by time and raw_pair so group by those

In [68]:
### for each raw_pair and time instance, select the max high price and minimum low price to identify opportunities for gain
df_pairs_grouped_time = df_pairs.groupby(['__time','raw_pair']).agg({'price_high':['count', 'max'], 'price_low':['min']})

In [70]:
df_pairs_grouped_time.to_csv('grouped_pairs_time_for_filter_count.csv')

In [161]:
#### look at the new index levels
df_pairs_grouped_time.columns.levels

FrozenList([['price_high', 'price_low'], ['count', 'max', 'min']])

In [72]:
#### filter out opportunities by limiting it to only rows where the raw pair and time
#### instance appears more than once otherwise there's no opportunity
mask_count = df_pairs_grouped_time[("price_high", "count")]>1

In [ ]:
#### narrow down the df by keeping only time and raw pair rows that happen more than once

In [84]:
greater_than_one_count = df_pairs_grouped_time[mask_count]

In [116]:
greater_than_one_reset = greater_than_one_count.reset_index()

In [121]:
new_df = pd.DataFrame(greater_than_one_reset.to_records())

In [125]:
new_df.columns = ['index', 'time', 'raw_pair', 'count', 'max_high_price', 'min_low_price']

In [127]:
new_df['price_differential'] = new_df['max_high_price'] - new_df['min_low_price']

In [142]:
#### create arb opp to denote the arbitrage opportunity when it exists to join it back to
#### original dataframe
new_df['time'] = new_df['time'].astype(str)
new_df['arb_opp'] = new_df['raw_pair'] + " " + new_df['time'].astype(str)

In [144]:
df['arb_opp'] = df['raw_pair'] +" "+ df['__time']

In [145]:
result = pd.merge(new_df, df, how='inner', left_on = 'arb_opp', right_on = 'arb_opp')

In [ ]:
#### look at how many opportunities there are 

In [150]:
result.shape

(450984, 32)

#### Appendix 

In [42]:
### get max and min of the group bys 
price_highs_grouped_by_ticker_time = df_pairs.groupby(['__time','raw_pair0', 'raw_pair1']).agg({'price_high': ['max', 'min']})

In [45]:
price_highs_grouped_by_ticker_time.to_csv('price_highs_grouped_by_ticker_time.csv')

In [41]:
price_lows_grouped_by_ticker_time = df_pairs.groupby(['__time','raw_pair0', 'raw_pair1']).agg({'price_low': ['max', 'min']})

In [44]:
price_lows_grouped_by_ticker_time.to_csv('price_lows_grouped_by_ticker_time.csv')

In [12]:
df_pairs.groupby(['raw_pair0', 'raw_pair1', 'marketplace']).mean().to_csv('grouped_by_ticker_marketplace.csv')

In [13]:
df_pairs.groupby(['raw_pair0', 'raw_pair1', 'marketplace', 'year']).mean().to_csv('grouped_by_ticker_marketplace_year.csv')

In [14]:
grouped_tickers = df_pairs.groupby(['raw_pair0', 'raw_pair1'], as_index = False).mean()

In [16]:
grouped_tickers.raw_pair0.value_counts()

USDT        29
BTC         28
ETH         21
BNB         19
XRP         18
            ..
POT          1
QNT          1
GUP          1
LINKDOWN     1
XTP          1
Name: raw_pair0, Length: 568, dtype: int64

In [101]:
tickers = ['USDT', 'BTC', 'ETH', 'BNB', 'XRP', 'TRX', 'LINK', 'EOS', 'NEO', 'LTC', 'BCH', 'ZEC', 'XLM', 'ALGO', 'USD']
grouped_tickers[grouped_tickers['raw_pair0'].str.contains('|'.join(tickers)) | grouped_tickers['raw_pair1'].str.contains('|'.join(tickers))].to_csv('above_10_occurances.csv')

#### Variables to arbitrage
#1. Day of the week
#2. Month
#3. Day of the month
#4. Year
#5. Daylight saving zones
#6. pairs
#7. marketplace


In [21]:
df['marketplace']=df['marketplace'].astype('category').cat.codes
df['raw_pair0']=df['raw_pair0'].astype('category').cat.codes
df['raw_pair1']=df['raw_pair1'].astype('category').cat.codes

In [18]:
tickers = ['BTC/ETH', 'ETH/BTC']
df_pairs[df_pairs['raw_pair'].str.contains('|'.join(tickers))]

,__time,marketplace,raw_pair,price_open,price_high,price_low,price_close,price_vwap,trade_count,volume,...,month,day,year,time,24_hour_time,time6,time7,time8,raw_pair0,raw_pair1
290,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,binance,ETH/BTC,0.034882,0.035487,0.034882,0.035241,0.035231,178627,258855.6700,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ETH,BTC
295,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,bitfinex,ETH/BTC,0.034883,0.035449,0.034883,0.035260,0.035287,4760,13520.2160,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ETH,BTC
301,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,bitstamp,ETH/BTC,0.034899,0.035459,0.034899,0.035265,0.035266,749,3694.1702,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ETH,BTC
304,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,coinbase,ETH/BTC,0.034879,0.035632,0.034879,0.035260,0.035348,6876,13549.5490,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ETH,BTC
309,Fri Mar 15 2019 18:00:00 GMT-0600 (Mountain Da...,gemini,ETH/BTC,0.035520,0.035570,0.034660,0.034660,0.035487,290,2350.9844,...,Mar,15,2019,18:00:00,GMT-0600,(Mountain,Daylight,Time),ETH,BTC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166286,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,coinbase,ETH/BTC,0.031334,0.031334,0.030991,0.031229,0.031127,23428,17303.4510,...,Mar,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),ETH,BTC
1166725,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,gemini,ETH/BTC,0.031347,0.031347,0.030983,0.031188,0.031129,513,1544.7507,...,Mar,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),ETH,BTC
1166728,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,kraken,ETH/BTC,0.031349,0.031349,0.030949,0.031169,0.031138,4662,7074.7330,...,Mar,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),ETH,BTC
1166739,Sat Mar 13 2021 17:00:00 GMT-0700 (Mountain St...,poloniex,ETH/BTC,0.031344,0.031344,0.031040,0.031194,0.031156,4818,11730.2690,...,Mar,13,2021,17:00:00,GMT-0700,(Mountain,Standard,Time),ETH,BTC
